In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle


In [2]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## Preprocess the data
### Drop irrelevant columns, such as RowNumber, CustomerId, Surname
data = data.drop(["RowNumber", "CustomerId", "Surname"], axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [4]:
##Encode categorical varaiables from Female and Male to 0 and 1
label_encoder_gender = LabelEncoder()
data["Gender"] = label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [5]:
## One hot encode the Geography column
from sklearn.preprocessing import OneHotEncoder

# oneHotEncoder = OneHotEncoder()
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[["Geography"]])
geo_encoder


<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [6]:
## 
onehot_encoder_geo.get_feature_names_out(["Geography"])


array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
geo_encoder_df = pd.DataFrame(
    geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(["Geography"])
)
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [8]:
##Combine one hot encoder columns with the original data
data = pd.concat([data.drop("Geography", axis=1), geo_encoder_df], axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [9]:
## Save the encoders and scaler
with open("label_encoder_gender.pkl", "wb") as file:
    pickle.dump(label_encoder_gender, file)

with open("onehot_encoder_geo.pkl", "wb") as file:
    pickle.dump(onehot_encoder_geo, file)



In [10]:
## Divide the dataset into independent and dependant features
data.head()
X = data.drop("Exited", axis=1)
y = data["Exited"]

## Split the data in training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
X_train


array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [11]:
X_test

array([[-5.12501721e-01,  9.09111664e-01, -6.77299309e-01, ...,
        -1.02020406e+00,  1.73668197e+00, -5.63491843e-01],
       [-2.36046598e-01,  9.09111664e-01,  3.84298354e-01, ...,
         9.80196059e-01, -5.75810666e-01, -5.63491843e-01],
       [-4.61306328e-01, -1.09997489e+00,  4.80807232e-01, ...,
        -1.02020406e+00, -5.75810666e-01,  1.77464858e+00],
       ...,
       [ 8.59534812e-01, -1.09997489e+00,  7.70333868e-01, ...,
         9.80196059e-01, -5.75810666e-01, -5.63491843e-01],
       [ 4.70449825e-01,  9.09111664e-01, -9.66825944e-01, ...,
         9.80196059e-01, -5.75810666e-01, -5.63491843e-01],
       [-1.84851205e-01,  9.09111664e-01, -1.73715981e-03, ...,
        -1.02020406e+00,  1.73668197e+00, -5.63491843e-01]])

In [12]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler, file)

In [13]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [14]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
(X_train.shape[1])

12

In [16]:
# Build the ANN model
model = Sequential(
    [
        Dense(
            64, activation="relu", input_shape=(X_train.shape[1],)
        ),  # First Hidden Layer
        Dense(32, activation="relu"),  # Second Hidden Layer
        Dense(1, activation="sigmoid"),  # Output Layer
    ]
)

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
import tensorflow 
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)


In [19]:
## Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
## Setup the tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = "logs/fit"
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [22]:
## Setup early stopping can stop the training if the model is not learning anymore
early_stopping_callback = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)


In [23]:
history=model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100, callbacks=[early_stopping_callback, tensorflow_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7382 - loss: 0.5257 - val_accuracy: 0.8255 - val_loss: 0.4007
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 951us/step - accuracy: 0.8342 - loss: 0.3974 - val_accuracy: 0.8500 - val_loss: 0.3616
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 826us/step - accuracy: 0.8554 - loss: 0.3582 - val_accuracy: 0.8565 - val_loss: 0.3458
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step - accuracy: 0.8576 - loss: 0.3506 - val_accuracy: 0.8575 - val_loss: 0.3444
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 829us/step - accuracy: 0.8609 - loss: 0.3329 - val_accuracy: 0.8565 - val_loss: 0.3426
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8617 - loss: 0.3353 - val_accuracy: 0.8600 - val_loss: 0.3403
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 828us/step - accuracy: 0.8666 - loss: 0.3269 - val_accuracy: 0.8585 - val_loss: 0.3407
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step - accuracy: 0.8566 - loss: 0.340

In [24]:
model.save("model.h5")

In [25]:
%load_ext tensorboard

In [26]:
%tensorboard --logdir logs/fit